In [2]:
from sqlalchemy import create_engine
import psycopg2 #driver to run query for PostgreSQL from Python

In [3]:
#connect to an existing database
conn = psycopg2.connect(database="roadtrips", user="sashakapralov")

#open a cursor to perform database operations
cur = conn.cursor()

In [4]:
import json

In [7]:
import csv

In [10]:
import geojson

In [14]:
#convert csv file with coordinates, bearing and depth values of tower used in geotagging code to GeoJSON
features = []
with open('detection_input_one.csv', 'r') as inf:
    reader = csv.reader(inf)
    next(inf)
    for lat,lon,bearing,depth in reader:
        lat, lon = map(float, (lat, lon))
        features.append(
            geojson.Feature(
                geometry = geojson.Point((lon*(-1.),lat)),
                properties = {
                    'bearing': bearing,
                    'depth': depth
                }
            )
        )

collection = geojson.FeatureCollection(features)
with open("one_tower.geojson", "w") as outf:
    outf.write('%s' % collection)

In [19]:
#run this line if Postgres returns any error on a query before re-running fixed query
cur.execute("rollback")

In [15]:
#execute a command: this creates a new table
#Then commit the changes to the database
cur.execute("DROP TABLE IF EXISTS detected_tower")
cur.execute("CREATE TABLE detected_tower (id SERIAL PRIMARY KEY, bearing FLOAT8 NOT NULL, depth FLOAT8 NOT NULL, latlon GEOMETRY)")
conn.commit()

In [16]:
#First delete the existing contents of this table in case we want to run the code multiple times.
cur.execute("DELETE FROM detected_tower")

#read GeoJSON file and load data into database table
with open('one_tower.geojson') as f:
    data = json.load(f)
    #loop through list of dictionaries of frames and their coordinates
    for frame in data['features']:
        bearing = frame['properties']['bearing']
        depth = frame['properties']['depth']
        #json.dumps is needed below to create double quotes inside geometry dictionary
        latlon = json.dumps(frame['geometry'])
        cur.execute("""INSERT INTO detected_tower (bearing, depth, latlon) VALUES ({}, {}, ST_SetSRID(ST_GeomFromGeoJSON('{}'),4326))""".format(bearing, depth, latlon))
conn.commit()

In [32]:
#SQL query to find OSM-tagged (true) coordinates of water tower that was used in geotagging code
cur.execute("SELECT nneighbors.frameno, nneighbors.frame_loc_geog, nneighbors.tower_id, ST_AsText(nneighbors.tower_loc_geog), nneighbors.dist \
            FROM nneighbors RIGHT JOIN detected_tower \
            ON nneighbors.frame_loc_geog = detected_tower.latlon")
results = cur.fetchall()

In [33]:
[frame[3] for frame in results]

['POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)',
 'POINT(-90.6424566 38.5982481)']

In [40]:
true_lon_lat = ''.join(c for c in results[0][3] if (c.isdigit() | (c in ['-','.',' '])))
true_lon, true_lat = map(float,true_lon_lat.split())
true_lon, true_lat

(-90.6424566, 38.5982481)

In [41]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from gmplot import GoogleMapPlotter
from random import random

In [186]:
class CustomGoogleMapPlotter(GoogleMapPlotter):
    def __init__(self, center_lat, center_lng, zoom, apikey='',
                 map_type='satellite'):
        if apikey == '':
            try:
                with open('apikey.txt', 'r') as apifile:
                    apikey = apifile.readline()
            except FileNotFoundError:
                pass
        super().__init__(center_lat, center_lng, zoom, apikey)

        self.map_type = map_type
        assert(self.map_type in ['roadmap', 'satellite', 'hybrid', 'terrain'])

    def write_map(self,  f):
        f.write('\t\tvar centerlatlng = new google.maps.LatLng(%f, %f);\n' %
                (self.center[0], self.center[1]))
        f.write('\t\tvar myOptions = {\n')
        f.write('\t\t\tzoom: %d,\n' % (self.zoom))
        f.write('\t\t\tcenter: centerlatlng,\n')

        # Change this line to allow different map types
        f.write('\t\t\tmapTypeId: \'{}\'\n'.format(self.map_type))

        f.write('\t\t};\n')
        f.write(
            '\t\tvar map = new google.maps.Map(document.getElementById("map_canvas"), myOptions);\n')
        f.write('\n')

    def color_scatter(self, lats, lngs, values=None, colormap='coolwarm',
                      size=None, marker=False, s=None, **kwargs):
        def rgb2hex(rgb):
            """ Convert RGBA or RGB to #RRGGBB """
            rgb = list(rgb[0:3]) # remove alpha if present
            rgb = [int(c * 255) for c in rgb]
            hexcolor = '#%02x%02x%02x' % tuple(rgb)
            return hexcolor

        if values is None:
            colors = [None for _ in lats]
        else:
            cmap = plt.get_cmap(colormap)
            norm = Normalize(vmin=min(values), vmax=max(values))
            scalar_map = ScalarMappable(norm=norm, cmap=cmap)
            colors = [rgb2hex(scalar_map.to_rgba(value)) for value in values]
        for lat, lon, c in zip(lats, lngs, colors):
            self.scatter(lats=[lat], lngs=[lon], c=c, size=size, marker=marker,
                         s=s, **kwargs)

    def cam_tower_scatter(self, lats, lngs, color=None, size=None, marker=True, c=None, s=None, symbol='o',
                          tower_color=None, true_tower_color=None, **kwargs):
        color = color or c
        size = size or s or 40
        kwargs["color"] = color
        kwargs["size"] = size
        settings = self._process_kwargs(kwargs)
        for lat, lng in zip(lats[:-2], lngs[:-2]):
            if marker:
                self.marker(lat, lng, settings['color'])
            else:
                self._add_symbol(Symbol(symbol, lat, lng, size), **settings)
        if marker:
            self.marker(lats[-2], lngs[-2], tower_color)
            tower_col = self.color_dict.get(color)
            tower_col = self.html_color_codes.get(color,color)
            self.points.append((lats[-2], lngs[-2], tower_col[1:], "no implementation"))
            
            if true_tower_color:            
                self.marker(lats[-1], lngs[-1], true_tower_color)
                true_tower_col = self.color_dict.get(color)
                true_tower_col = self.html_color_codes.get(color,color)
                self.points.append((lats[-1], lngs[-1], true_tower_col[1:], "no implementation"))

In [210]:
cam_lats = []
cam_lons = []
with open('./detection_input_one.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cam_lats.append(float(row['lat']))
        cam_lons.append(float(row['lon'])*(-1.))
        #print(row['lat'],row['lon'])
print('cam_lats:',cam_lats,'\ncam_lons:',cam_lons)

cam_lats: [38.596667, 38.597111, 38.597333, 38.597389, 38.597389, 38.597389, 38.597389, 38.597472, 38.597778] 
cam_lons: [-90.640028, -90.640417, -90.640639, -90.640694, -90.640694, -90.640694, -90.640694, -90.640806, -90.641111]


In [211]:
norm_lats = []
norm_lons = []
with open('./object_positions.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        norm_lats.append(float(row['latp1']))
        norm_lons.append(float(row['lonp1'])*(-1.))
print('norm_lats:',norm_lats,'\nnorm_lons:',norm_lons)

norm_lats: [38.595937138530665, 38.59573936425739, 38.59584943001999, 38.59568711062047, 38.59563378523053, 38.59563378523053, 38.59563378523053, 38.59561175655161, 38.595866091317724] 
norm_lons: [-90.63563457530068, -90.63628248210517, -90.63656824931394, -90.6367655787464, -90.63680418071672, -90.63680418071672, -90.63680418071672, -90.63699692716777, -90.63734404705237]


In [212]:
tower_lats = []
tower_lons = []
with open('./water_tower_detection_loop.csv','r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        tower_lats.append(float(row['lat']))
        tower_lons.append(float(row['lon'])*(-1.))
print('tower_lats:',tower_lats,'\ntower_lons:',tower_lons)

tower_lats: [38.5966613] 
tower_lons: [-90.63904579999999]


In [160]:
all_lats = cam_lats + tower_lats
all_lons = cam_lons + tower_lons

In [213]:
all_lats = cam_lats + norm_lats + tower_lats
all_lons = cam_lons + norm_lons + tower_lons

In [214]:
all_lats.append(true_lat)

In [215]:
all_lons.append(true_lon)

In [216]:
initial_zoom = 12
gmap = CustomGoogleMapPlotter(all_lats[0], all_lons[0], initial_zoom,
                              map_type='satellite')
gmap.cam_tower_scatter(all_lats,all_lons,alpha=0.5,c='red', marker=True, tower_color='blue', true_tower_color='yellow')
gmap.draw("tower_map_with_true_v26.html")

#### Plot points produced by Intersection() function

In [208]:
int_lats = []
int_lons = []
with open('./intersections.csv','r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        int_lats.append(float(row[0]))
        int_lons.append(float(row[1])*(-1.))
print('int_lats:',int_lats,'\nint_lons:',int_lons)

int_lats: [38.59635110191371, 38.59629564405737, 38.59639739589488, 38.59642141323996, 38.59642141323996, 38.59642141323996, 38.596447877657866, 38.59639387542463, 38.595679303410876, 38.59659444835876, 38.59668615394061, 38.59668615394061, 38.59668615394061, 38.59674873715421, 38.59651699763621, 38.59716252227377, 38.597202055601045, 38.597202055601045, 38.597202055601045, 38.5971640718349, 38.59680979859447, 38.59716666157824, 38.596355221904275, 38.59704512823787, 38.59596792937662, 38.59704512823787, 38.59596792937662, 38.59704512823787, 38.59596792937662, 38.59362132847227, 38.59652243424516] 
int_lons: [-90.63812644126169, -90.63779261090339, -90.63840510937729, -90.63854968256209, -90.63854968256209, -90.63854968256209, -90.63870898598084, -90.63838391779288, -90.6361014408586, -90.6388599597903, -90.63913638767669, -90.63913638767669, -90.63913638767669, -90.63932503208751, -90.63862650028759, -90.6401712281184, -90.64027970316259, -90.64027970316259, -90.64027970316259, -90.64

In [209]:
initial_zoom = 12
gmap = CustomGoogleMapPlotter(int_lats[0], int_lons[0], initial_zoom,
                              map_type='satellite')
gmap.scatter(int_lats,int_lons,alpha=0.5,c='red',marker=True)
gmap.cam_tower_scatter(all_lats,all_lons,alpha=0.5,c='red', marker=True, tower_color='blue')
gmap.draw("ints_map_v6.html")

In [222]:
from math import radians, pi

In [223]:
bearing = 420
br1 = radians(bearing)
br1 - 2*pi

1.0471975511965983

In [ ]:
bearing = 0
br1 =